<a href="https://colab.research.google.com/github/thanit456/image_classification_playground/blob/master/easy_efficientnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Swish activation

In [0]:
from tensorflow.keras import backend as K

def swish_activation(x):
  return x*K.sigmoid(x)

# Inverted Residual Block

In [0]:
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, Add, GlobalAveragePooling2D, Reshape

def inverted_residual_block(x, expand=64, squeeze=16):
  block = Conv2D(expand, (1, 1), activation='relu')(x)
  block = DepthwiseConv2D((3, 3), activation='relu')(block)
  block = Conv2D(squeeze, (1, 1), activation='relu')(block)
  return Add()[block, x]

def se_block(x, filters, squeeze_ratio=0.25):
  x_ = GlobalAveragePooling2D()(x_)
  x_ = Reshape((1, 1, filters))(x_)
  squeeze_filters = max(1, int(filters * squeeze_ratio))
  x_ = Conv2D(squeeze_filters, activation='relu')(x_)
  x_ = Conv2D(filters, activation='sigmoid')(x_)
  return multiply()([x, x_])

In [3]:
argument_block = [
                  BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, 
                            output_filters=16, expand_ratio=1, id_skip=True, strides=[1, 1], se_ratio=0.25),
                  BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, 
                            output_filters=24, expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
                  BlockArgs(kernel_size=5, num_repeat=2, input_filters=24, 
                            output_filters=40, expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
                  BlockArgs(kernel_size=3, num_repeat=3, input_filters=40, 
                            output_filters=80, expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
                  BlockArgs(kernel_size=5, num_repeat=3, input_filters=80, 
                            output_filters=112, expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25),
                  BlockArgs(kernel_size=5, num_repeat=4, input_filters=112, 
                            output_filters=192, expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25)
                  BlockArgs(kernel_size=3, num_repeat=1, input_filters=192, 
                            output_filters=320, expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25)
]

SyntaxError: ignored

In [0]:
def mbConv_block(input_data, block_args):
  kernel_size = block_arg.kernel_size
  num_repeat = block_arg.num_repeat
  input_filters = block_arg.input_filters
  out_filters = out_filters.kernel_size
  expand_ratio = block_arg.expand_ratio
  id_skip = block_arg.id_skip
  strides = block_arg.strides
  se_ratio = block_arg.se_ratio

  # Expansion phase
  expanded_filters = input_filters * expand_ratio
  x = Conv2D(expanded_filters, 1, padding='same', use_bias=False)(input_data)
  x = BatchNormalization()(x)
  x = Activation(swish_activation)(x)
  
  # Depthwise convolution phas
  x = DepthwiseConv2D(kernel_size, strides, padding='same', use_bias=False)(x)
  x = BatchNormalization()(x)
  x = Activation(swish_activation)(x)
  
  # Squeeze and excitation phase
  se = GlobalAveragePooling2D()(x)
  se = Reshape((1, 1, expanded_filters))(se)
  squeezed_filters = max(1, int(input_filters * se_ratio))
  se = Conv2D(squeezed_filters, 1, activation=swish_activation, padding='same')(se)
  x = multiply([x, se])
  
  # Output phase
  x = Conv2D(out_filters, 1, padding='same', use_bias=False)
  x = BatchNormalization()(x)
  return x
